In [ ]:
# bc of pydantic discrepancies, need to run in harbor environment instead of asap2025

# Imports

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from pathlib import Path
import harbor.analysis.cross_docking as cd
from importlib import reload
reload(cd)
import multiprocessing as mp
from pydantic import BaseModel
import plotly.graph_objects as go

In [ ]:
# load data
fred_data = pd.read_csv('/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20250113_fred_docking/rmsd_csvs/20250122_combined_results_with_data.csv', index_col=0)
posit_data = pd.read_csv("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20240424_multi_pose_docking_cross_docking/results_csvs/20240503_combined_results_with_data.csv", index_col=0)
figure_path = Path("figures")
fred_data['Engine'] = 'FRED'
posit_data['Engine'] = 'POSIT'
results_df = pd.concat([fred_data, posit_data])

In [ ]:
fred_data_sp = fred_data.sort_values(['Pose_ID']).groupby(['Query_Ligand', 'Reference_Structure']).head(1)
posit_data_sp = posit_data.sort_values(['Pose_ID']).groupby(['Query_Ligand', 'Reference_Structure']).head(1)

In [ ]:
n_per_splits = np.arange(1, 21)
n_per_splits = np.concatenate((n_per_splits, np.arange(25, 206, 20)))

In [ ]:
fred_data_sp

In [ ]:
reload(cd)
evaluators = []
# Set up pose selectors
pose_selectors = [
    cd.PoseSelector(name="Default", variable="Pose_ID", number_to_return=1)
]
# Set up dataset splits
dataset_splits = []
dataset_splits.extend(
    [
        cd.SimilaritySplit(
            variable="Tanimoto",
            n_splits=1,
            n_per_split=-1,
            threshold=i,
            include_similar=False,
            higher_is_more_similar=True,
        )
    for i in np.linspace(0, 1, 20)]
)
# structure_choices = []
# structure_choices.extend(
#         [
#             cd.StructureChoice(
#                 name="ECFP4_Similarity",
#                 variable="Tanimoto",
#                 higher_is_better=False,
#                 number_to_return=n_per_split,
#             )
#             for n_per_split in n_per_splits
#         ]
#     )

# Add scorers
scorers = [
    cd.Scorer(
        name="POSIT_Probability",
        variable="docking-confidence-POSIT",
        number_to_return=1,
    ),
    cd.Scorer(
        name="RMSD", variable="RMSD", higher_is_better=False, number_to_return=1
    ),
]
rmsd_evaluator = cd.BinaryEvaluation(variable="RMSD", cutoff=2)

In [ ]:
for scorer in scorers:
    for split in dataset_splits:
        for selector in pose_selectors:
            # for structure_choice in structure_choices:
                evaluators.append(
                    cd.Evaluator(
                        pose_selector=selector,
                        dataset_split=split,
                        # structure_choice=structure_choice
                        scorer=scorer,
                        evaluator=rmsd_evaluator,
                        groupby=["Query_Ligand"],
                        n_bootstraps=(100),
                    )
                )

In [ ]:
len(evaluators)

In [ ]:
results = []
from tqdm import tqdm
for evaluator in tqdm(evaluators):
    results.append(cd.Results.calculate_result(evaluator=evaluator, df=posit_data_sp))
    results.append(cd.Results.calculate_result(evaluator=evaluator, df=fred_data_sp))

In [ ]:
# pull out every other result
posit_results = results[::2]
fred_results = results[1::2]

In [ ]:
pr_df = cd.Results.df_from_results(posit_results)
fr_df = cd.Results.df_from_results(fred_results)

In [ ]:
pr_df['Engine'] = 'POSIT'
fr_df['Engine'] = 'FRED'

In [ ]:
results_df = pd.concat([pr_df, fr_df])

In [ ]:
results_df.N_Per_Split = results_df.N_Per_Split.astype(int)
results_df.sort_values(["Split", "Score", "PoseSelection", "StructureChoice", "StructureChoice_Choose_N", "N_Per_Split"], inplace=True)
results_df["Error_Lower"] = results_df["Fraction"] - results_df["CI_Lower"]
results_df["Error_Upper"] = results_df["CI_Upper"] - results_df["Fraction"]

# Plotting Variables

In [ ]:
large_font = 24
small_font = 18 
labels = {"Fraction": "<b> Fraction of Poses Docked < 2Å from Reference </b>",
               "N_Per_Split": "<b> Total Number of Reference Structures Available to Use </b>",
          "Date": "Temporally Ordered",
                   "Random": "Randomly Shuffled",
          "Similarity_Threshold": "<b> Tanimoto Similarity </b>",
          }
update_layout_dict = dict(xaxis=dict(title_font=dict(size=large_font), 
                            color='black', 
                            ),
                  yaxis=dict(range=(0,1),  
                      title_font=dict(size=large_font), 
                             color='black', 
                             ))

In [ ]:
def update_traces(fig):
    for trace in fig.data:
        if trace.name is None:
            continue
        trace.name = trace.name.replace("_", " ")
        trace.name = trace.name.replace("Split", "")
        trace.name = trace.name.replace(", ", " | ")
        trace.name = trace.name.replace("RMSD", "RMSD (Positive Control)")
        for k, v in labels.items():
            trace.name = trace.name.replace(k, v)
    return fig

In [ ]:
def hex_to_rgb(hex_color: str) -> tuple:
    hex_color = hex_color.lstrip("#")
    if len(hex_color) == 3:
        hex_color = hex_color * 2
    return int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)

In [ ]:
def rgb_to_rgba(rgb_str, alpha):
    # Split the RGB string into its components
    rgb_values = rgb_str.strip('rgb()').split(',')
    
    # Extract individual RGB values and convert them to integers
    r, g, b = map(int, rgb_values)
    
    # Construct the RGBA string
    rgba_str = f"rgba({r}, {g}, {b}, {alpha})"
    
    return rgba_str

In [ ]:
def line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [ ]:
results_df

In [ ]:
fig = line(data_frame=results_df, 
              x="Similarity_Threshold", 
              y="Fraction", 
              color="Engine",
              line_dash="Score",
              error_y="Error_Upper", 
              error_y_minus="Error_Lower", 
           error_y_mode="band",
              template="simple_white",
           # facet_col="Engine",
              # symbol="Score", 
              height=600, 
              width=800,
              # log_x=True,
              # color_discrete_sequence=px.colors.qualitative.Dark2,
              labels = labels
              )
fig.update_layout(
    font=dict(size=small_font, 
              family='Arial'
              ),
    legend=dict(title="<b> Score Function | Dataset Split </b>", 
                              x=0.4, y=0.1, 
                              traceorder='reversed', 
                              title_font_size=large_font, 
                              font_color='black'),
    **update_layout_dict)
fig.update_yaxes(tickvals=np.arange(0, 1.1, 0.1))
fig.update_layout(xaxis_tickformat = "0.1r", xaxis_tickvals=np.linspace(0, 1, 11))
fig = update_traces(fig)
fig.show()
fig.write_image(figure_path / "08_20250211_chemical_similarity.svg")
fig.write_image(figure_path / "08_20250211_chemical_similarity.png")

# Chemical Similarity Plots

In [ ]:
# get just chemical pair data
similarity_data = pd.read_csv("/Users/alexpayne/Scientific_Projects/asapdiscovery-sars-retrospective/data/all_tanimoto.csv")

In [ ]:
fig = px.ecdf(similarity_data, x="Tanimoto", color="Similarity Metric", template="simple_white", height=600, width=800)

In [ ]:
fig.show()
fig.write_image(figure_path / "08_20250211_chemical_similarity_ecdf.svg")
fig.write_image(figure_path / "08_20250211_chemical_similarity_ecdf.png")

In [ ]:
boxplot = px.box(similarity_data, y="Similarity Metric", x="Tanimoto", template="simple_white", height=600, width=800)
boxplot.show()

In [ ]:
boxplot = px.box(similarity_data[similarity_data["Similarity Metric"] == "ECFP4"], 
                 y="Similarity Metric", 
                 x="Tanimoto", 
                 template="simple_white",
                 # log_x=True,
                 height=200, 
                 width=800)
boxplot.update_layout(xaxis_tickformat = "0.1r")
boxplot.show()
boxplot.write_image(figure_path / "08_20250211_chemical_similarity_ecfp4_boxplot.svg")
boxplot.write_image(figure_path / "08_20250211_chemical_similarity_ecfp4_boxplot.png")

In [ ]:
from scipy.stats import rv_continuous, ecdf

In [ ]:
ecfp4 = ecdf(similarity_data[similarity_data["Similarity Metric"] == "ECFP4"]["Tanimoto"])

In [ ]:
ecfp4.cdf.plot()

# I'd like to pull out the Tanimoto similarity values for specific intervals of the data

In [ ]:
def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
idxs = [find_nearest_idx(ecfp4.cdf.probabilities, val) for val in [0,0.25,0.5,0.75,1]]

In [ ]:
quantiles = ecfp4.cdf.quantiles[idxs]
probs = ecfp4.cdf.probabilities[idxs]

In [ ]:
px.scatter(x=quantiles, y=probs)

# I'd like to get the per-ligand values

## i.e. how many structures are available for each ligand at each Tanimoto similarity level

In [ ]:
ecfp4_data = similarity_data[similarity_data["Similarity Metric"] == "ECFP4"]

In [ ]:
n_structures_per_lig = []
n_mols = ecfp4_data["Mol1"].nunique()
tanimoto_vals = np.linspace(0, 1, 11)
for t in tanimoto_vals:
    count_df = ecfp4_data.groupby("Mol1").apply(lambda x: x[x["Tanimoto"] <= t].count())[["Mol1"]]
    count_df = count_df.rename(columns={"Mol1": "Count"})
    count_df["Tanimoto"] = t
    count_df["Fraction"] = count_df["Count"] / n_mols
    n_structures_per_lig.append(count_df)

In [ ]:
n_structures_per_lig_df = pd.concat(n_structures_per_lig)

In [ ]:
fig = px.box(n_structures_per_lig_df, x="Tanimoto", y="Fraction", template="simple_white", height=600, width=800)
fig.write_image(figure_path / "08_20250211_chemical_similarity_ecfp4_structures_per_ligand_fraction.svg")
fig.write_image(figure_path / "08_20250211_chemical_similarity_ecfp4_structures_per_ligand_fraction.png")

In [ ]:
fig = px.box(n_structures_per_lig_df, x="Tanimoto", y="Count", template="simple_white", height=600, width=800)
fig.write_image(figure_path / "08_20250211_chemical_similarity_ecfp4_structures_per_ligand_count.svg")
fig.write_image(figure_path / "08_20250211_chemical_similarity_ecfp4_structures_per_ligand_count.png")